In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
old_v = tf.compat.v1.logging.get_verbosity()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# 导入 MINST 数据集
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)
tf.compat.v1.logging.set_verbosity(old_v)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
batch_size = 100
time_step =28 # 时间步（每个时间步处理图像的一行）
data_length = 28 # 每个时间步输入数据的长度（这里就是图像的宽度）
learning_rate = 0.01

In [3]:
# 定义占位符
X_ = tf.placeholder(tf.float32, [None, 784])
Y_ = tf.placeholder(tf.int32, [None, 10])

# dynamic_rnn的输入数据(batch_size, max_time, ...)
inputs = tf.reshape(X_, [-1, time_step, data_length])

# 验证集
validate_data = {X_: mnist.validation.images, Y_: mnist.validation.labels}
# 测试集
test_data = {X_: mnist.test.images, Y_: mnist.test.labels}

# 定义一个两层的GRU模型
gru_layers = rnn.MultiRNNCell([rnn.GRUCell(num_units=num) for num in [100, 100]], state_is_tuple=True)

outputs, h_ = tf.nn.dynamic_rnn(gru_layers, inputs, dtype=tf.float32)
output = tf.layers.dense(outputs[:, -1, :], 10) #获取GRU网络的最后输出状态

# 定义交叉熵损失函数和优化器
loss = tf.losses.softmax_cross_entropy(onehot_labels=Y_, logits=output)
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# 计算准确率
accuracy = tf.metrics.accuracy(labels=tf.argmax(Y_, axis=1), predictions=tf.argmax(output, axis=1))[1]

## 初始化变量
sess = tf.Session()
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)

for step in range(3000):
    # 获取一个batch的训练数据
    train_x, train_y = mnist.train.next_batch(batch_size)
    _, loss_ = sess.run([train_op, loss], {X_: train_x, Y_: train_y})
    
    # 在验证集上计算准确率
    if step % 100 == 0:
        val_acc = sess.run(accuracy, feed_dict=validate_data)
        print('step:', step,'train loss: %.4f' % loss_, '| val accuracy: %.2f' % val_acc)
    
## 计算测试集史上的准确率
test_acc = sess.run(accuracy, feed_dict=test_data)
print('test loss: %.4f' % test_acc)

W0816 15:02:08.115901 140215968261952 module_wrapper.py:136] From /usr/local/python3/lib/python3.6/site-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 15:02:08.121917 140215968261952 deprecation.py:323] From <ipython-input-3-b0516593e217>:14: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
W0816 15:02:08.123775 140215968261952 deprecation.py:323] From <ipython-input-3-b0516593e217>:14: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0816 15:02:08.124845 140215968261952 deprecation.py:323

step: 0 train loss: 2.3052 | val accuracy: 0.13
step: 100 train loss: 0.1519 | val accuracy: 0.53
step: 200 train loss: 0.1007 | val accuracy: 0.68
step: 300 train loss: 0.0438 | val accuracy: 0.75
step: 400 train loss: 0.0623 | val accuracy: 0.80
step: 500 train loss: 0.1239 | val accuracy: 0.83
step: 600 train loss: 0.0623 | val accuracy: 0.85
step: 700 train loss: 0.0691 | val accuracy: 0.86
step: 800 train loss: 0.0951 | val accuracy: 0.88
step: 900 train loss: 0.0745 | val accuracy: 0.89
step: 1000 train loss: 0.1131 | val accuracy: 0.89
step: 1100 train loss: 0.1958 | val accuracy: 0.90
step: 1200 train loss: 0.0453 | val accuracy: 0.91
step: 1300 train loss: 0.0631 | val accuracy: 0.91
step: 1400 train loss: 0.0495 | val accuracy: 0.92
step: 1500 train loss: 0.0844 | val accuracy: 0.92
step: 1600 train loss: 0.0420 | val accuracy: 0.92
step: 1700 train loss: 0.0085 | val accuracy: 0.93
step: 1800 train loss: 0.0670 | val accuracy: 0.93
step: 1900 train loss: 0.2119 | val accurac